In [9]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
import time
import ssl
from PIL import Image
from urllib.request import urlretrieve
import urllib
import concurrent.futures
from pygeckodriver import geckodriver_path #! pip install pygeckodriver
import os

import random
import string

def generate_random_string(length):
    # Define the characters to choose from
    characters = string.ascii_letters + string.digits
    # Generate a random string
    random_string = ''.join(random.choice(characters) for i in range(length))
    return random_string
import pytesseract # !pip install pytesseract
# download: tesseract-ocr-w64-
pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

# Specify the path to GeckoDriver
#gecko_driver_path = 'C:/selenium/geckodriver.exe'  # Update this to your GeckoDriver path, just download GeckoDriver from github
def get_title(cedula, max_tries=20):
    # Set up the Service for GeckoDriver
    service = Service(geckodriver_path)
    #service = Service()
    # Set up the Firefox driver
    driver = webdriver.Firefox(service=service)
    #driver = webdriver.Firefox(executable_path=geckodriver_path)

    n_tries = 0
    success = False
    while success == False:

        driver.get("https://www.senescyt.gob.ec/web/guest/consultas")
        time.sleep(2)

        # check for outer html
        # captcha: <div class="ui-panelgrid-cell gri-height ui-grid-col-6"><img id="formPrincipal:capimg" src="/consulta-titulos-web/consulta-titulos-web/../Captcha.jpg;jsessionid=nm576ojCuQtaHCAkhLO3H6rt7HqEqDwB7X-TirO2.srvprouioct26" style="width:100% !important;"></div>


        img = driver.find_element(By.ID, 'formPrincipal:capimg')
        src = img.get_attribute("src")

        # Create an unverified SSL context
        ctx = ssl._create_unverified_context()
        # Create an opener with our SSL context
        opener = urllib.request.build_opener(urllib.request.HTTPSHandler(context=ctx))

        # Install the opener globally
        urllib.request.install_opener(opener)
        captcha_name = generate_random_string(12) + ".png"

        try:
            os.remove(captcha_name)
        except:
            None

        urlretrieve(src, captcha_name)

        # leo imagen captcha
        img = Image.open(captcha_name)
        img = img.convert("RGB")
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Declara ubicación tesseract
        captcha_response = pytesseract.image_to_string(img)

        # mando cedula de interés
        driver.find_element(By.ID, 'formPrincipal:identificacion').send_keys(cedula)
        # lleno captcha
        driver.find_element(By.ID, 'formPrincipal:captchaSellerInput').send_keys(captcha_response)
        # click en buscar
        time.sleep(2)
        driver.find_element(By.ID, 'formPrincipal:boton-buscar').click()

        try:
            src_print = driver.find_element(By.ID, 'formPrincipal:btnInfoConsulta')
            success = True
        except:
            success = False

        n_tries += 1

        if n_tries >= max_tries:
            success = True
        try:
            os.remove(captcha_name)
        except:
            None

    #<span class="ui-messages-error-detail">Caracteres incorrectos</span>

    if success:
        identification = driver.find_element(By.ID,"formPrincipal:j_idt36").text
        names = driver.find_element(By.ID,"formPrincipal:j_idt38").text
        gender = driver.find_element(By.ID,"formPrincipal:j_idt40").text
        nationality = driver.find_element(By.ID,"formPrincipal:j_idt42").text

        # Print extracted information
        res = dict()

        res["Identification"] = identification
        res["Names"] = names
        res["Gender"] = gender
        res["Nationality"] = gender


        names = ["Título","Institución","Tipo","Reconocido Por","Número de Registro", "Fecha de Registro", "Área o Campo de Conocimiento", "Observación"]
        # Find the table body by ID
        tbody = driver.find_element(By.ID, "formPrincipal:j_idt44:0:tablaAplicaciones_data")
        i = 0
        # Iterate over each row in the tbody
        for row in tbody.find_elements(By.TAG_NAME, "tr"):
            # Each cell ('td') in the row
            for cell in row.find_elements(By.TAG_NAME, "td"):
                # Assuming each cell has a span followed by the actual text
                cell_data = cell.text
                res[names[i]] = cell_data
                i += 1

    driver.close()



    return res

In [10]:
# Correr una sola cédula
get_title("1804468799")

{'Identification': 'Identificación:',
 'Names': 'Nombres:',
 'Gender': 'Género:',
 'Nationality': 'Género:',
 'Título': 'ECONOMISTA',
 'Institución': 'PONTIFICIA UNIVERSIDAD CATOLICA DEL ECUADOR',
 'Tipo': 'Nacional',
 'Reconocido Por': '',
 'Número de Registro': '1027-15-1390810',
 'Fecha de Registro': '2015-07-28',
 'Área o Campo de Conocimiento': 'CIENCIAS SOCIALES, EDUCACION COMERCIAL Y DERECHO',
 'Observación': ''}

In [11]:
task_durations = ["1804468799","1804468799","1804468799","1804468799"]
 
# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit the tasks
    result = list(executor.map(get_title, task_durations))

In [14]:
result

[{'Identification': 'Identificación:',
  'Names': 'Nombres:',
  'Gender': 'Género:',
  'Nationality': 'Género:',
  'Título': 'ECONOMISTA',
  'Institución': 'PONTIFICIA UNIVERSIDAD CATOLICA DEL ECUADOR',
  'Tipo': 'Nacional',
  'Reconocido Por': '',
  'Número de Registro': '1027-15-1390810',
  'Fecha de Registro': '2015-07-28',
  'Área o Campo de Conocimiento': 'CIENCIAS SOCIALES, EDUCACION COMERCIAL Y DERECHO',
  'Observación': ''},
 {'Identification': 'Identificación:',
  'Names': 'Nombres:',
  'Gender': 'Género:',
  'Nationality': 'Género:',
  'Título': 'ECONOMISTA',
  'Institución': 'PONTIFICIA UNIVERSIDAD CATOLICA DEL ECUADOR',
  'Tipo': 'Nacional',
  'Reconocido Por': '',
  'Número de Registro': '1027-15-1390810',
  'Fecha de Registro': '2015-07-28',
  'Área o Campo de Conocimiento': 'CIENCIAS SOCIALES, EDUCACION COMERCIAL Y DERECHO',
  'Observación': ''},
 {'Identification': 'Identificación:',
  'Names': 'Nombres:',
  'Gender': 'Género:',
  'Nationality': 'Género:',
  'Título': '